### **CS 3300 - Final Project**

###### Paige Rosynek & Xavier Robbins

#### **Import Libraries**

In [53]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from nlp_functions import normalize_text
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
import spacy

#### **Data Cleaning**

**Import the Data**

In [54]:
trending_df = pd.read_csv('../data/US_youtube_trending_data.csv')

In [55]:
# load data using Python JSON module
with open('../data/US_category_id.json','r') as f:
    data = json.loads(f.read())
    
category_df = pd.json_normalize(data, record_path=['items'])

In [56]:
trending_df.head()

,video_id,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,view_count,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description
0,3C66w5Z0ixs,I ASKED HER TO BE MY GIRLFRIEND...,2020-08-11T19:20:14Z,UCvtRTOMP2TqYqu51xNrqAzg,Brawadis,22,2020-08-12T00:00:00Z,brawadis|prank|basketball|skits|ghost|funny vi...,1514614,156908,5855,35313,https://i.ytimg.com/vi/3C66w5Z0ixs/default.jpg,False,False,SUBSCRIBE to BRAWADIS ▶ http://bit.ly/Subscrib...
1,M9Pmf9AB4Mo,Apex Legends | Stories from the Outlands – “Th...,2020-08-11T17:00:10Z,UC0ZV6M2THA81QT9hrVWJG3A,Apex Legends,20,2020-08-12T00:00:00Z,Apex Legends|Apex Legends characters|new Apex ...,2381688,146739,2794,16549,https://i.ytimg.com/vi/M9Pmf9AB4Mo/default.jpg,False,False,"While running her own modding shop, Ramya Pare..."
2,J78aPJ3VyNs,I left youtube for a month and THIS is what ha...,2020-08-11T16:34:06Z,UCYzPXprvl5Y-Sf0g4vX-m6g,jacksepticeye,24,2020-08-12T00:00:00Z,jacksepticeye|funny|funny meme|memes|jacksepti...,2038853,353787,2628,40221,https://i.ytimg.com/vi/J78aPJ3VyNs/default.jpg,False,False,I left youtube for a month and this is what ha...
3,kXLn3HkpjaA,XXL 2020 Freshman Class Revealed - Official An...,2020-08-11T16:38:55Z,UCbg_UMjlHJg_19SZckaKajg,XXL,10,2020-08-12T00:00:00Z,xxl freshman|xxl freshmen|2020 xxl freshman|20...,496771,23251,1856,7647,https://i.ytimg.com/vi/kXLn3HkpjaA/default.jpg,False,False,Subscribe to XXL → http://bit.ly/subscribe-xxl...
4,VIUo6yapDbc,Ultimate DIY Home Movie Theater for The LaBran...,2020-08-11T15:10:05Z,UCDVPcEbVLQgLZX0Rt6jo34A,Mr. Kate,26,2020-08-12T00:00:00Z,The LaBrant Family|DIY|Interior Design|Makeove...,1123889,45802,964,2196,https://i.ytimg.com/vi/VIUo6yapDbc/default.jpg,False,False,Transforming The LaBrant Family's empty white ...


In [57]:
trending_df.describe()

,categoryId,view_count,likes,dislikes,comment_count
count,177390.000000,1.773900e+05,1.773900e+05,177390.000000,1.773900e+05
mean,18.840267,2.531660e+06,1.328712e+05,1718.693528,1.115995e+04
std,6.751101,7.038882e+06,4.096811e+05,9854.973492,8.487794e+04
min,1.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00
25%,17.000000,4.850082e+05,1.920925e+04,0.000000,1.359000e+03
50%,20.000000,9.883235e+05,4.441650e+04,186.000000,3.034000e+03
75%,24.000000,2.225834e+06,1.111325e+05,1009.000000,7.214000e+03
max,29.000000,2.777917e+08,1.602153e+07,879354.000000,6.738537e+06


In [58]:
trending_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177390 entries, 0 to 177389
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   video_id           177390 non-null  object
 1   title              177390 non-null  object
 2   publishedAt        177390 non-null  object
 3   channelId          177390 non-null  object
 4   channelTitle       177390 non-null  object
 5   categoryId         177390 non-null  int64 
 6   trending_date      177390 non-null  object
 7   tags               177390 non-null  object
 8   view_count         177390 non-null  int64 
 9   likes              177390 non-null  int64 
 10  dislikes           177390 non-null  int64 
 11  comment_count      177390 non-null  int64 
 12  thumbnail_link     177390 non-null  object
 13  comments_disabled  177390 non-null  bool  
 14  ratings_disabled   177390 non-null  bool  
 15  description        173416 non-null  object
dtypes: bool(2), int64(5)

In [59]:
category_df.head()

,kind,etag,id,snippet.title,snippet.assignable,snippet.channelId
0,youtube#videoCategory,IfWa37JGcqZs-jZeAyFGkbeh6bc,1,Film & Animation,True,UCBR8-60-B28hp2BmDPdntcQ
1,youtube#videoCategory,5XGylIs7zkjHh5940dsT5862m1Y,2,Autos & Vehicles,True,UCBR8-60-B28hp2BmDPdntcQ
2,youtube#videoCategory,HCjFMARbBeWjpm6PDfReCOMOZGA,10,Music,True,UCBR8-60-B28hp2BmDPdntcQ
3,youtube#videoCategory,ra8H7xyAfmE2FewsDabE3TUSq10,15,Pets & Animals,True,UCBR8-60-B28hp2BmDPdntcQ
4,youtube#videoCategory,7mqChSJogdF3hSIL-88BfDE-W8M,17,Sports,True,UCBR8-60-B28hp2BmDPdntcQ


In [60]:
category_df.describe()

,kind,etag,id,snippet.title,snippet.assignable,snippet.channelId
count,32,32,32,32,32,32
unique,1,32,32,31,2,1
top,youtube#videoCategory,IfWa37JGcqZs-jZeAyFGkbeh6bc,1,Comedy,False,UCBR8-60-B28hp2BmDPdntcQ
freq,32,1,1,2,17,32


In [61]:
category_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   kind                32 non-null     object
 1   etag                32 non-null     object
 2   id                  32 non-null     object
 3   snippet.title       32 non-null     object
 4   snippet.assignable  32 non-null     bool  
 5   snippet.channelId   32 non-null     object
dtypes: bool(1), object(5)
memory usage: 1.4+ KB


In [62]:
# convert type of column to make merge successful
category_df['id'] = category_df['id'].astype('int64')

In [63]:
trending_df['channelId'].value_counts()

UCDVYQ4Zhbm3S2dlz7P1GBDg    1180
UCWJ2lWNubArHWmf3FIHbfcQ    1056
UCpB959t8iPrxQWj7G6n0ctQ     595
UCqZQlzSHbVJrwrn5XvzrzcA     520
UCIPPMRA040LQr5QPyJEbmXA     512
                            ... 
UCHmv6QMb4xerJ8Tt5UBHjaw       1
UCfrSUhUYOUyZROqWZb-TphQ       1
UCiio0ydw439X13KyZgMIcHw       1
UCDi0kbBfktecByFXHMzFkww       1
UCD6ERRdXrF2IZ0R888G8PQg       1
Name: channelId, Length: 6838, dtype: int64

In [64]:
category_df['snippet.channelId'].value_counts()

UCBR8-60-B28hp2BmDPdntcQ    32
Name: snippet.channelId, dtype: int64

**Merge Datasets Into Single DataFrame**

In [65]:
df = pd.merge(trending_df, category_df, left_on=['categoryId'], right_on=['id'], how='inner')

In [66]:
df.shape

(177390, 22)

In [67]:
df.head()

,video_id,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,view_count,likes,...,thumbnail_link,comments_disabled,ratings_disabled,description,kind,etag,id,snippet.title,snippet.assignable,snippet.channelId
0,3C66w5Z0ixs,I ASKED HER TO BE MY GIRLFRIEND...,2020-08-11T19:20:14Z,UCvtRTOMP2TqYqu51xNrqAzg,Brawadis,22,2020-08-12T00:00:00Z,brawadis|prank|basketball|skits|ghost|funny vi...,1514614,156908,...,https://i.ytimg.com/vi/3C66w5Z0ixs/default.jpg,False,False,SUBSCRIBE to BRAWADIS ▶ http://bit.ly/Subscrib...,youtube#videoCategory,QMEBz6mxVdklVaq8JwesPEw_4nI,22,People & Blogs,True,UCBR8-60-B28hp2BmDPdntcQ
1,49Z6Mv4_WCA,i don't know what im doing anymore,2020-08-11T20:24:34Z,UCtinbF-Q-fVthA0qrFQTgXQ,CaseyNeistat,22,2020-08-12T00:00:00Z,[None],940036,87111,...,https://i.ytimg.com/vi/49Z6Mv4_WCA/default.jpg,False,False,ssend love to my sponsor; for a super Limited ...,youtube#videoCategory,QMEBz6mxVdklVaq8JwesPEw_4nI,22,People & Blogs,True,UCBR8-60-B28hp2BmDPdntcQ
2,nt3VVyv5pxQ,Try Not To Laugh Challenge #51,2020-08-11T17:00:31Z,UCYJPby9DRCteedh5tfxVbrw,Smosh Pit,22,2020-08-12T00:00:00Z,smosh|smosh pit|smosh games|funny|comedy,591837,44168,...,https://i.ytimg.com/vi/nt3VVyv5pxQ/default.jpg,False,False,You know what time it is— time to try not to l...,youtube#videoCategory,QMEBz6mxVdklVaq8JwesPEw_4nI,22,People & Blogs,True,UCBR8-60-B28hp2BmDPdntcQ
3,BNeDH6UTmXw,Ten Minutes with Tyler Cameron | Q&A,2020-08-11T22:00:05Z,UCMw7m-ScQ6jV1FQzQnn1y8Q,Tyler Cameron,22,2020-08-12T00:00:00Z,the bachelor|the bachelorette|Tyler c|Tyler Ca...,105955,4511,...,https://i.ytimg.com/vi/BNeDH6UTmXw/default.jpg,False,False,Come hang out me with me for 10 minutes where ...,youtube#videoCategory,QMEBz6mxVdklVaq8JwesPEw_4nI,22,People & Blogs,True,UCBR8-60-B28hp2BmDPdntcQ
4,gPdUslndvVI,Our Farm Got Destroyed.,2020-08-11T23:00:06Z,UCuxlXCfVyV-i5YLL30jkomw,Cole The Cornstar,22,2020-08-12T00:00:00Z,farming|family farm|agriculture|agriculture jo...,277338,37533,...,https://i.ytimg.com/vi/gPdUslndvVI/default.jpg,False,False,"Wind storm, rain, and lots of destruction; wel...",youtube#videoCategory,QMEBz6mxVdklVaq8JwesPEw_4nI,22,People & Blogs,True,UCBR8-60-B28hp2BmDPdntcQ


##### **Clean Merged Dataset**

In [68]:
category_col = df['snippet.title']
df = df.drop(labels=['snippet.channelId', 'id', 'snippet.assignable', 'etag', 'kind', 'categoryId', 'snippet.title', 'thumbnail_link'], axis=1)
df['category'] = category_col

**Convert Merged Dataframe Columns to Correct Type**

In [69]:
df['video_id'] = df['video_id'].astype('category')
df['title'] = df['title'].astype('category')
df['channelId'] = df['channelId'].astype('category')
df['channelTitle'] = df['channelTitle'].astype('category')
df['tags'] = df['tags'].astype('category')
df['description'] = df['description'].astype('category')
df['category'] = df['category'].astype('category')
df['publishedAt'] = pd.to_datetime(df['publishedAt'])
df['trending_date'] = pd.to_datetime(df['trending_date'])

In [70]:
df.head()

,video_id,title,publishedAt,channelId,channelTitle,trending_date,tags,view_count,likes,dislikes,comment_count,comments_disabled,ratings_disabled,description,category
0,3C66w5Z0ixs,I ASKED HER TO BE MY GIRLFRIEND...,2020-08-11 19:20:14+00:00,UCvtRTOMP2TqYqu51xNrqAzg,Brawadis,2020-08-12 00:00:00+00:00,brawadis|prank|basketball|skits|ghost|funny vi...,1514614,156908,5855,35313,False,False,SUBSCRIBE to BRAWADIS ▶ http://bit.ly/Subscrib...,People & Blogs
1,49Z6Mv4_WCA,i don't know what im doing anymore,2020-08-11 20:24:34+00:00,UCtinbF-Q-fVthA0qrFQTgXQ,CaseyNeistat,2020-08-12 00:00:00+00:00,[None],940036,87111,1860,7052,False,False,ssend love to my sponsor; for a super Limited ...,People & Blogs
2,nt3VVyv5pxQ,Try Not To Laugh Challenge #51,2020-08-11 17:00:31+00:00,UCYJPby9DRCteedh5tfxVbrw,Smosh Pit,2020-08-12 00:00:00+00:00,smosh|smosh pit|smosh games|funny|comedy,591837,44168,409,2652,False,False,You know what time it is— time to try not to l...,People & Blogs
3,BNeDH6UTmXw,Ten Minutes with Tyler Cameron | Q&A,2020-08-11 22:00:05+00:00,UCMw7m-ScQ6jV1FQzQnn1y8Q,Tyler Cameron,2020-08-12 00:00:00+00:00,the bachelor|the bachelorette|Tyler c|Tyler Ca...,105955,4511,69,673,False,False,Come hang out me with me for 10 minutes where ...,People & Blogs
4,gPdUslndvVI,Our Farm Got Destroyed.,2020-08-11 23:00:06+00:00,UCuxlXCfVyV-i5YLL30jkomw,Cole The Cornstar,2020-08-12 00:00:00+00:00,farming|family farm|agriculture|agriculture jo...,277338,37533,197,3666,False,False,"Wind storm, rain, and lots of destruction; wel...",People & Blogs


In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 177390 entries, 0 to 177389
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype              
---  ------             --------------   -----              
 0   video_id           177390 non-null  category           
 1   title              177390 non-null  category           
 2   publishedAt        177390 non-null  datetime64[ns, UTC]
 3   channelId          177390 non-null  category           
 4   channelTitle       177390 non-null  category           
 5   trending_date      177390 non-null  datetime64[ns, UTC]
 6   tags               177390 non-null  category           
 7   view_count         177390 non-null  int64              
 8   likes              177390 non-null  int64              
 9   dislikes           177390 non-null  int64              
 10  comment_count      177390 non-null  int64              
 11  comments_disabled  177390 non-null  bool               
 12  ratings_disabled   177390 non-

<mark>ADD MORE COLUMNS? WHAT TO DO WITH TEXT COLUMNS -> slam it all in the df</mark>

tfidvectorizer

- morning, afternoon, night ?
- day of the week
- month ?
- days between trending videos
- days between publish & trending date
- number of days trending

**Upload & Trending Day of Week**

- 0 : monday, 1 : tuesday, ...

In [72]:
df['dayofweek_upload'] = df['publishedAt'].dt.dayofweek
df['dayofweek_trending'] = df['trending_date'].dt.dayofweek

Get DFs of first video id occurrence and last video id occurrence 

In [73]:
first_video_occ_df = df.drop_duplicates('video_id', keep='first').reset_index(drop=True)
last_video_occ_df = df.drop_duplicates('video_id', keep='last').reset_index(drop=True)

Create new df of first video occurences and add columns from last occurence df 

In [74]:
new_df = first_video_occ_df.rename(columns={'trending_date':'start_trend_date', 
                                            'view_count':'start_view_count', 
                                            'likes':'start_likes', 
                                            'dislikes':'start_dislikes', 
                                            'comment_count':'start_comment_count'})

Add last occurence columns to new DF for data that changes

In [75]:
new_df['end_trend_date'] = last_video_occ_df['trending_date']
new_df['end_view_count'] = last_video_occ_df['view_count']
new_df['end_likes'] = last_video_occ_df['likes']
new_df['end_dislikes'] = last_video_occ_df['dislikes']
new_df['end_comment_count'] = last_video_occ_df['comment_count']

Reset index of df and print head elements

In [76]:
new_df = new_df.sort_values('video_id').reset_index(drop=True)
new_df.head()

,video_id,title,publishedAt,channelId,channelTitle,start_trend_date,tags,start_view_count,start_likes,start_dislikes,...,ratings_disabled,description,category,dayofweek_upload,dayofweek_trending,end_trend_date,end_view_count,end_likes,end_dislikes,end_comment_count
0,--14w5SOEUs,Migos - Avalanche,2021-06-10 16:00:00+00:00,UCGIelM2Dj3zza3xyV3pL3WQ,MigosVEVO,2021-06-11 00:00:00+00:00,Migos|Avalanche|Quality|Control|Music/Motown|R...,1076217,122830,867,...,False,Music video by Migos performing Avalanche. Qua...,Music,3,4,2021-06-17 00:00:00+00:00,1942190,60716,694,3835
1,--2O86Z0hsM,MY TESLA PAYS FOR ITSELF,2022-03-09 23:19:08+00:00,UCXJEvxZSozjAAqhbMfhIArA,jf.okay,2022-03-11 00:00:00+00:00,[None],466980,16481,0,...,False,NaN,Entertainment,2,4,2022-03-16 00:00:00+00:00,1531226,61093,0,4334
2,--40TEbZ9Is,Supporting Actress in a Comedy: 73rd Emmys,2021-09-20 01:03:32+00:00,UClBKH8yZRcM4AsRjDVEdjMg,Television Academy,2021-09-21 00:00:00+00:00,[None],527742,6299,286,...,False,Hannah Waddingham wins the Emmy for Supporting...,Entertainment,0,1,2021-09-26 00:00:00+00:00,327437,18966,422,1561
3,--5-brQiQFg,Washington Commanders vs. San Francisco 49ers ...,2022-12-25 00:30:17+00:00,UCDVYQ4Zhbm3S2dlz7P1GBDg,NFL,2022-12-26 00:00:00+00:00,[None],1280997,14603,0,...,False,Check out our other channels:NFL Mundo https:/...,Sports,6,0,2022-12-31 00:00:00+00:00,20078922,463039,0,21311
4,--DKkzWVh-E,Why Retaining Walls Collapse,2021-12-07 13:00:00+00:00,UCMOqf8ab-42UUQIdVoKwjlQ,Practical Engineering,2021-12-08 00:00:00+00:00,retaining wall|New Jersey highway|Direct Conne...,320130,18445,147,...,False,One of the most important (and innocuous) part...,Education,1,2,2021-12-15 00:00:00+00:00,786433,46899,0,2556


In [79]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32444 entries, 0 to 32443
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype              
---  ------               --------------  -----              
 0   video_id             32444 non-null  category           
 1   title                32444 non-null  category           
 2   publishedAt          32444 non-null  datetime64[ns, UTC]
 3   channelId            32444 non-null  category           
 4   channelTitle         32444 non-null  category           
 5   start_trend_date     32444 non-null  datetime64[ns, UTC]
 6   tags                 32444 non-null  category           
 7   start_view_count     32444 non-null  int64              
 8   start_likes          32444 non-null  int64              
 9   start_dislikes       32444 non-null  int64              
 10  start_comment_count  32444 non-null  int64              
 11  comments_disabled    32444 non-null  bool               
 12  ratings_disabled  

**Number of Days Trending**

In [87]:
new_df['number_days_trend'] = new_df['end_trend_date'] - new_df['start_trend_date']
new_df['number_days_trend'] = new_df['number_days_trend'].dt.days
new_df.head()

,video_id,title,publishedAt,channelId,channelTitle,start_trend_date,tags,start_view_count,start_likes,start_dislikes,...,description,category,dayofweek_upload,dayofweek_trending,end_trend_date,end_view_count,end_likes,end_dislikes,end_comment_count,number_days_trend
0,--14w5SOEUs,Migos - Avalanche,2021-06-10 16:00:00+00:00,UCGIelM2Dj3zza3xyV3pL3WQ,MigosVEVO,2021-06-11 00:00:00+00:00,Migos|Avalanche|Quality|Control|Music/Motown|R...,1076217,122830,867,...,Music video by Migos performing Avalanche. Qua...,Music,3,4,2021-06-17 00:00:00+00:00,1942190,60716,694,3835,6
1,--2O86Z0hsM,MY TESLA PAYS FOR ITSELF,2022-03-09 23:19:08+00:00,UCXJEvxZSozjAAqhbMfhIArA,jf.okay,2022-03-11 00:00:00+00:00,[None],466980,16481,0,...,NaN,Entertainment,2,4,2022-03-16 00:00:00+00:00,1531226,61093,0,4334,5
2,--40TEbZ9Is,Supporting Actress in a Comedy: 73rd Emmys,2021-09-20 01:03:32+00:00,UClBKH8yZRcM4AsRjDVEdjMg,Television Academy,2021-09-21 00:00:00+00:00,[None],527742,6299,286,...,Hannah Waddingham wins the Emmy for Supporting...,Entertainment,0,1,2021-09-26 00:00:00+00:00,327437,18966,422,1561,5
3,--5-brQiQFg,Washington Commanders vs. San Francisco 49ers ...,2022-12-25 00:30:17+00:00,UCDVYQ4Zhbm3S2dlz7P1GBDg,NFL,2022-12-26 00:00:00+00:00,[None],1280997,14603,0,...,Check out our other channels:NFL Mundo https:/...,Sports,6,0,2022-12-31 00:00:00+00:00,20078922,463039,0,21311,5
4,--DKkzWVh-E,Why Retaining Walls Collapse,2021-12-07 13:00:00+00:00,UCMOqf8ab-42UUQIdVoKwjlQ,Practical Engineering,2021-12-08 00:00:00+00:00,retaining wall|New Jersey highway|Direct Conne...,320130,18445,147,...,One of the most important (and innocuous) part...,Education,1,2,2021-12-15 00:00:00+00:00,786433,46899,0,2556,7


In [86]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32444 entries, 0 to 32443
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype              
---  ------               --------------  -----              
 0   video_id             32444 non-null  category           
 1   title                32444 non-null  category           
 2   publishedAt          32444 non-null  datetime64[ns, UTC]
 3   channelId            32444 non-null  category           
 4   channelTitle         32444 non-null  category           
 5   start_trend_date     32444 non-null  datetime64[ns, UTC]
 6   tags                 32444 non-null  category           
 7   start_view_count     32444 non-null  int64              
 8   start_likes          32444 non-null  int64              
 9   start_dislikes       32444 non-null  int64              
 10  start_comment_count  32444 non-null  int64              
 11  comments_disabled    32444 non-null  bool               
 12  ratings_disabled  

##### **Extract Words from Text Columns**

- columns: title, tags, description

In [77]:
lang_model = spacy.load("en_core_web_sm")

In [78]:
# for title in df['title']:
#     print(normalize_text(title, lang_model, lemmatizing=True, stop_words=True))